# N(utrient)-P(hytoplankton)-Z(ooplankton)-D(etritus) Model

### A toy interactive model of ocean ecosystem dynamics

---
<center>
Riley X. Brady
</center>

<center>
riley.brady@colorado.edu
</center>

---

**References:**

1. J.L. Sarmiento and N. Gruber (2006). Ocean Biogeochemical Dynamics. Chapter 4: "Production."

2. A.M. Edwards (2001). Adding Detritus to a Nutrient–Phytoplankton–Zooplankton Model :A Dynamical-Systems Approach. J Plankton Res.

---

## Summary

We can create a reduced model of a complex lower-trophic ocean ecosystem with a few differential equations. Here, we choose to model four variables: an arbitrary nutrient concentration (generally thought of as a macronutrient such as nitrate), a phytoplankton concentration (maybe a diatom), a zooplankton concentration (with option to change the type of zooplankter), and a detritus concentration (waste products).

In reality, we are just modeling a finite reservoir of nitrate, and considering how it gets redistributed around the ecosystem, given a few initial conditions and parameter settings. In other words, we aren't explicitly modeling phytoplankton cell count or biomass, but rather tracking where the nitrate goes as it is incorporated into organic matter via photosynthesis, or consumed by zooplankton.

Differential equations (DE's) are complex things to deal with. In a model like this, we have four DE's interacting with one another, because the rate of change of the given population (nutrient, phytoplankton, zooplankton, or detritus) is dependent on the current state of the other three populations. Thus, it is a lot easier to discretize a model into time steps, and reduce our DE's into algebraic equations that may be solved in reference to the current state of the system.

Here, I use an explicit time-differencing scheme (forward Euler method) to model this simple ocean ecosystem.

---

## Differential Equations Contributing to the Model
The four raw DE's are as follows (where N is our nutrients, P is our phytoplankton, Z is our zooplankton, and D is our detritus):

$\frac{dN}{dt} = -V_{m}\left(\frac{N}{K_{N}+N}\right)f(I_{0})P~+~\alpha R_{m}\left(1-e^{-\lambda P}\right)Z~+~\epsilon P~+~gZ~+~\phi D$

$\frac{dP}{dt} = V_{m}\left(\frac{N}{K_{N} + N}\right)f(I_{0})P~-~R_{m}\left(1 - e^{-\lambda P}\right)Z~-~\epsilon P~-~rP$

$\frac{dZ}{dt} = \beta R_{m}\left(1-e^{-\lambda P}\right)Z~-~gZ$

$\frac{dD}{dt} = rP~+~(1 - \alpha - \beta)R_{m}\left(1 - e^{-\lambda P}\right)Z~-~\phi D$

---

### Terms

#### Bulk Terms

If you look closely at each DE, you note that these are simply source (+) minus sink (-) equations. This simple model only has a few nitrogen exchange processes:

$V_{m}\left(\frac{N}{K_{N} + N}\right)f(I_{0})P$ : Phytoplankton grazing term. How much inorganic nitrogen are they taking up?

$R_{m}\left(1-e^{-\lambda P}\right)Z$ : Zooplankton grazing term. How much nitrogen are they taking up after consuming phytoplankton and releasing some as waste? The $\beta$ coefficient represents the proportion taken up into zooplankton organic matter; the $\alpha$ coefficient is that which is dissolved back into nutrients (perhaps from urine); and (1-$\alpha$-$\beta$) is that which is excreted as fecal pellets (to the detritus compartment).

$\epsilon P$ : How much nitrogen is being returned to the pool from phytoplankton death?

$gZ$ : How much nitrogen is being returned to the pool from zooplankton death?

$rP$ : How much nitrogen is being respired by phytoplankton into detritus?

#### Phytoplankton Terms

$V_{m}$ : Maximum growth rate of an individual plankter (div per day). This value is dependent on temperature, via a lab-derived equation for diatoms: $V_{m} = a\cdot b^{T}$, with $a=0.6d^{-1}$, $b=1.066$, and $c=1(degC)^{-1}$.

$K_{N}$ : Half-saturation constant for nitrogen uptake ($\mu molNl^{-1}$). This is the nitrogen concentration at which the phytoplankton growth rate is at half its maximum value.

$f_{0}$ : Light intensity (0 to 1 weighting function). This is a simple parameterization of a more complex hyperbolic term that uses a similar term to $K_{N}$.

$\epsilon$ : Phytoplankton death rate (cells per day).

$r$ : Respiration rate (per day).

#### Zooplankton Terms

$R_{m}$ : Maximum grazing rate of zooplankton on phytoplankton (cells per day).

$\lambda$ : Grazing constant ($\mu molNl^{-1}$).

$\beta$ : Proportion of assimilated nitrogen by zooplankton. In other words, when they graze upon a phytoplankter, how efficient are they at taking up the nitrogen? (dimensionless)

$\alpha$ : Proportion of nitrogen taken up by zooplankton that returns to the environment as dissolved nutrients (urine?).

$g$ : Zooplankton death rate (critters per day).

#### Detritus Terms
$\phi$ : The remineralization rate of detritus back into dissolved nutrients (per day).

---
### Fixed Values/Initial Conditions

|                 Parameter                |   Symbol   |   Default Value   |
|:----------------------------------------:|:----------:|:-----------------:|
| Ambient Temperature                      |      T     |      15 degC      |
| Half-saturation constant for $N$ uptake  |   $K_{N}$  |  1$\mu$mol per L  |
| Maximum Grazing Rate                     |   R$_{m}$  |     1 $d^{-1}$    |
| Zooplankton Death Rate                   |      g     |    0.2$d^{-1}$    |
| Zooplankton Grazing Constant             |  $\lambda$ | 0.2$\mu$mol per L |
| Phytoplankton Death Rate                 | $\epsilon$ |    0.1$d^{-1}$    |
| Proportional Light Intensity             |   f$_{0}$  |        0.25       |
| Zooplankton Dissolved Excretion Fraction |  $\alpha$  |        0.3        |
| Zooplankton Assimilation Efficiency      |   $\beta$  |        0.6        |
| Phytoplankton Respiration Rate           |      r     |        0.15       |
| Detritus Remineralization Rate           |   $\phi$   |    0.4 $d^{-1}$   |

In [ ]:
#### Code patched by Wing-Ho Ko, 2025-01-28
# Now works with latest (as of writing 3.6.2) version of bokeh

In [ ]:
import numpy as np

import bokeh
from bokeh import events
from bokeh.io import output_notebook, show, output_file, save
from bokeh.layouts import column, row, layout
from bokeh.models import RadioButtonGroup, Slider, ColumnDataSource, CustomJS
from bokeh.plotting import figure

In [ ]:
# Allow Bokeh to be utilized inline with Jupyter.
output_notebook()

In [ ]:
#### Create collection of widgets

# Nutrient Initial Conditions
nut_slider = Slider(start = 0, end = 10, value = 4, step = 0.5, title = "Initial Nutrient Concentration")

# Phytoplankton Initial Conditions
phyto_slider = Slider(start = 0, end = 10, value = 2.5, step = 0.5, title = "Initial Phytoplankton Concentration")

# Zooplankton Initial Conditions
zoo_slider = Slider(start = 0, end = 10, value = 1.5, step = 0.5, title = "Initial Zooplankton Concentration")

# Detritus Initial Conditions
det_slider = Slider(start = 0, end = 10, value = 0, step = 0.5, title = "Initial Detritus Concentration")

# Ambient Temperature
temp_slider = Slider(start = 0, end = 25, value = 15, step = 1, title = "Water Temperature (degC)")

# Phytoplankton Death Rate
pdeath_slider = Slider(start = 0, end = 0.5, value = 0.1, step = 0.05, title = "Phytoplankton Natural Death Rate (per day)")

# Zooplankton Death Rate
zdeath_slider = Slider(start = 0, end = 0.5, value = 0.2, step = 0.05, title = "Zooplankton Natural Death Rate (per day)")

# Light Intensity
light_slider = Slider(start = 0, end = 1, value = 0.25, step = 0.05, title = "Proportional Light Intensity")

# Zooplankton Species (for grazing)
zoo_species = RadioButtonGroup(labels=["Cladoceran", "Copepod", "Mysid", "Rotifer"], active=2)

In [ ]:
# Creating and initializing source data

NUM_STEPS = 150
dt = 1

cmap = ["#bebada", "#8dd3c7", "#fb8072", "#e5d8bd"]
labels = ["Nutrients", "Phytoplankton", "Zooplankton", "Detritus"]

source = ColumnDataSource({
        '0'    : np.zeros(NUM_STEPS),
        't'    : dt * np.arange(0, NUM_STEPS),
        'N'    : np.zeros(NUM_STEPS),
        'P'    : np.zeros(NUM_STEPS),
        'Z'    : np.zeros(NUM_STEPS),
        'D'    : np.zeros(NUM_STEPS),
        'NP'   : np.zeros(NUM_STEPS),
        'NPZ'  : np.zeros(NUM_STEPS),
        'NPZD' : np.zeros(NUM_STEPS)
})

In [ ]:
# TIME SERIES PLOT
plot = figure(
    outer_width=450, outer_height=300,
    toolbar_location="right", tools = "save", 
    x_range=(source.data["t"][0], source.data["t"][-1]), y_range=(0, 10),
    x_axis_label="Model days", y_axis_label="Concentration (μmol·N·L⁻¹)",
    title="N-P-Z-D Time Series"
)

plot.line(x='t', y='N', source=source, line_width=3, color=cmap[0], legend_label=labels[0])
plot.line(x='t', y='P', source=source, line_width=3, color=cmap[1], legend_label=labels[1])
plot.line(x='t', y='Z', source=source, line_width=3, color=cmap[2], legend_label=labels[2])
plot.line(x='t', y='D', source=source, line_width=3, color=cmap[3], legend_label=labels[3])

In [ ]:
# STACKED BAR PLOT
plot2 = figure(
    outer_width=450, outer_height=300,
    toolbar_location="right", tools = "save",
    x_range=(source.data["t"][0], source.data["t"][-1]), y_range=(0, 10),
    x_axis_label="Model days", y_axis_label="Concentration (μmol·N·L⁻¹)",
    title="Nutrient Distribution"
)

# Plot data
plot2.vbar(x='t', top='N', source=source, width=1, color=cmap[0])
plot2.vbar(x='t', bottom='N', top='NP', source=source, width=1, color=cmap[1])
plot2.vbar(x='t', bottom='NP', top='NPZ', source=source, width=1, color=cmap[2])
plot2.vbar(x='t', bottom='NPZ', top='NPZD', source=source, width=1, color=cmap[3])

In [ ]:
callback = CustomJS(
    args=dict(
        source=source, plot=plot, plot2=plot2,
        nut=nut_slider, phyto=phyto_slider, zoo=zoo_slider, det=det_slider, 
        temperature=temp_slider, p_death=pdeath_slider, z_death = zdeath_slider, light=light_slider,
        zooSpecies = zoo_species
    ), code="""

    // Ingest main model data for modification
    var data = source.data;
    var t_arr = data['t'];
    var N = data['N'];
    var P = data['P'];
    var Z = data['Z'];
    var D = data['D'];

    var NP = data['NP'];
    var NPZ = data['NPZ'];
    var NPZD = data['NPZD'];

    var NUM_STEPS = t_arr.length;

    // Parameters
    var Kn = 1; // Anything that is fixed and unable to be modified just gets the same value as the default view.
    var g = z_death.value; // Anything that will be a slider or button gets this JS call.
    var lambda_Z = 0.2;
    var epsilon = p_death.value;
    var f = light.value;
    var dt = 1;
    var T = temperature.value;

    // Detritus-Related Parameters
    var alpha = 0.3;
    var beta = 0.6;
    var r = 0.15;
    var phi = 0.4;

    // Calculate Maximum Grazing Rate
    var a  = 0.6;
    var b  = 1.066;
    var Vm = a * Math.pow(b, T);

    // Zooplankton Species (impacts grazing rate)
    // Need to use IF statements for the button widget.
    var entry = zooSpecies.active;
    if (entry === 0) {
        var Rm = 1.6;
    } else if (entry === 1) {
        var Rm = 1.8;
    } else if (entry === 2) {
        var Rm = 1;
    } else {
        var Rm = 2;
    }

    // Initial Conditions with modifications allowed
    var N_0 = nut.value;
    var P_0 = phyto.value;
    var Z_0 = zoo.value;
    var D_0 = det.value;

    // Insert Initial Values for model
    N[0]    = N_0;
    P[0]    = P_0;
    Z[0]    = Z_0;
    D[0]    = D_0;
    NP[0] = N_0 + P_0;
    NPZ[0] = N_0 + P_0 + Z_0;
    NPZD[0] = N_0 + P_0 + Z_0 + D_0;

    // Run Model
    for (let i = 1; i < NUM_STEPS; i++) {
        var t = i - 1;

        // Common terms
        var gamma_N   = N[t] / (Kn + N[t])
        var zoo_graze = Rm * (1 - Math.exp(-lambda_Z * P[t])) * Z[t]

        // Equation calculations for model
        N[i] = dt * (-Vm*gamma_N*f*P[t] + alpha*zoo_graze + epsilon*P[t] + g*Z[t] + phi*D[t]) + N[t];
        P[i] = dt * (Vm*gamma_N*f*P[t] - zoo_graze - epsilon*P[t] - r*P[t]) + P[t];
        Z[i] = dt * (beta*zoo_graze - g*Z[t]) + Z[t];
        D[i] = dt * (r*P[t] + (1-alpha-beta)*zoo_graze - phi*D[t]) + D[t];

        // Sum Variables
        NP[i] = N[i] + P[i];
        NPZ[i] = N[i] + P[i] + Z[i];
        NPZD[i] = N[i] + P[i] + Z[i] + D[i];
    }

    plot.y_range.end = NPZD[0];
    plot2.y_range.end = NPZD[0];

    source.change.emit();
    plot.change.emit();
    plot2.change.emit();
    """
)

In [ ]:
nut_slider.js_on_change('value', callback)
phyto_slider.js_on_change('value', callback)
zoo_slider.js_on_change('value', callback)
det_slider.js_on_change('value', callback)
temp_slider.js_on_change('value', callback)
pdeath_slider.js_on_change('value', callback)
zdeath_slider.js_on_change('value', callback)
light_slider.js_on_change('value', callback)
zoo_species.js_on_change('active', callback)

In [ ]:
out = layout([
    [nut_slider, phyto_slider], 
    [zoo_slider, det_slider], 
    [pdeath_slider, zdeath_slider], 
    [light_slider, temp_slider], 
    [zoo_species],
    [plot, plot2]
])

In [ ]:
show(out)

In [ ]:
output_file('NPZD_bokeh.html')
save(out)